In [ ]:
import pandas as pd
from tqdm.notebook import tqdm
import numpy as np
from sklearn.model_selection import train_test_split

SEQ_LEN = 50
# INPUT_SIZE = 3 * 25  # x,y,z,vis * 25 points , removed vis

#remove head landmarks - # TODO FIX outside this func
INPUT_SIZE = 3 * (25-11)
head = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
str_head = [str(h) for h in head]

# y: ndarray (1200,)
# x: ndarray (1200, 50, 100)
def read_raw_data(score_path, data_path):
    score_df = pd.read_excel(score_path, engine="openpyxl")
    score_df = score_df[score_df['frames'] <= SEQ_LEN]

    data_df = pd.read_csv(data_path)
    amount_of_shots = len(score_df)
    print('all examples:', amount_of_shots)

    single_shot_x = np.zeros((SEQ_LEN + 1, INPUT_SIZE))
    x = np.zeros((amount_of_shots, SEQ_LEN + 1, INPUT_SIZE))
    y = []
    shot_index = 0

    for _, row in tqdm(score_df.iterrows()):
        frames = row['frames']
        score = row['score']
        name = row['name']

        for i in range(0, frames):
            frame_name = name + '_{}'.format(i)
            shot_data = []
            for k, v in data_df[data_df['name'] == frame_name].iteritems():
                if k == 'name' or k.endswith('vis'):
                    # TODO FIX outside this func
                    continue
                if any(k.split('_')[0] == h for h in str_head):
                    continue
                shot_data.append(v.values[0])
            single_shot_x[i] = np.array(shot_data)

        if shot_index == amount_of_shots:
            break

        y.append(score)
        x[shot_index] = np.nan_to_num(single_shot_x)
        shot_index += 1

    return x, np.array(y)


x, y = read_raw_data(score_path='labels/fco_score2.xlsx', data_path='labels/fco_data2.csv')

In [ ]:
print('x shape', x.shape)
print('y shape', y.shape)
print(SEQ_LEN + 1, INPUT_SIZE)

import matplotlib.pyplot as plt
plt.hist(y)
plt.show()